In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=c179a76193135f4e000dfc89d259495cc440674bbe62277b3b550d04d7f4c978
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
#spark kodu database e gotrup calistirdigi icin diger veri tabanlarina gore daha hizli calisiyor

In [3]:
#spark verileri tablo olarak degil satir olarak okudugu icin satirlari birlestiriyoruz

In [4]:
from pyspark import SparkConf, SparkContext #conf ve icerik dosyalari
import collections#toplamak icin

sc = SparkContext()
rdd = sc.parallelize([4, 5, 2, 2])#rdd sparkin degisken adi  paralel hesaplama

sq = rdd.map(lambda x: x*x) #yukardaki verilerin karesini aldik

print(sq.collect())#hesap sonuclarini topluyor

sc.stop()#1 kere calistir

[16, 25, 4, 4]


In [ ]:
sc.stop()

In [20]:
# veri setini ice aktariyoruz

In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("veriokuma") \
    .getOrCreate()

df = spark.read.csv("churn.csv", header=True, inferSchema=True)

### EDA

In [9]:
df.show(10)

+---+----------------+----+--------------+---------------+-----+---------+-----+
|_c0|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|Churn|
+---+----------------+----+--------------+---------------+-----+---------+-----+
|  0|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|    1|
|  1|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|    1|
|  2|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|    1|
|  3|   Phillip White|42.0|       8010.76|              0| 6.71|     10.0|    1|
|  4|  Cynthia Norton|37.0|       9191.58|              0| 5.56|      9.0|    1|
|  5|Jessica Williams|48.0|      10356.02|              0| 5.12|      8.0|    1|
|  6|     Eric Butler|44.0|      11331.58|              1| 5.23|     11.0|    1|
|  7|   Zachary Walsh|32.0|       9885.12|              1| 6.92|      9.0|    1|
|  8|     Ashlee Carr|43.0|       14062.6|              1| 5.46|     11.0|    1|
|  9|  Jennifer Lynch|40.0| 

In [10]:
value_counts = df.groupBy("Churn").count()
value_counts.show()

+-----+-----+
|Churn|count|
+-----+-----+
|    1|  150|
|    0|  750|
+-----+-----+



In [11]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Churn: integer (nullable = true)



In [12]:
df.describe().show()

+-------+------------------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+
|summary|               _c0|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|              Churn|
+-------+------------------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+
|  count|               900|          900|              900|              900|               900|              900|               900|                900|
|   mean|             449.5|         NULL|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|0.16666666666666666|
| stddev|259.95191863111916|         NULL|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.7648355920350969| 0.3728852122772358|
|    min|                 0|   Aaron King|             22.0|          

In [16]:
from pyspark.sql.functions import col, sum

null_counts = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
null_counts.show()

+---+-----+---+--------------+---------------+-----+---------+-----+
|_c0|Names|Age|Total_Purchase|Account_Manager|Years|Num_Sites|Churn|
+---+-----+---+--------------+---------------+-----+---------+-----+
|  0|    0|  0|             0|              0|    0|        0|    0|
+---+-----+---+--------------+---------------+-----+---------+-----+



### MODELLING

In [19]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import GBTClassifier
# Sayısal özellik sütunlarını seçin
numeric_cols = ['Age','Total_Purchase','Account_Manager','Years','Num_Sites']

assembler = VectorAssembler(inputCols=numeric_cols, outputCol="features")

# Veri kümesini özellik vektörleri ve hedef sütunu olarak hazırlayın
data = assembler.transform(df).select("features", "Churn")

# Eğitim ve test verisi olarak bölün
train_data, test_data = data.randomSplit([0.7, 0.3], seed=42)

# GBTClassifier modelini oluşturun
gbt = GBTClassifier(labelCol="Churn", featuresCol="features")

# Pipeline oluşturun
pipeline = Pipeline(stages=[gbt])

# Modeli eğitin
model = pipeline.fit(train_data)

# Test verileri üzerinde tahmin yapın
predictions = model.transform(test_data)

# Tahminleri değerlendirin
evaluator = BinaryClassificationEvaluator(labelCol="Churn")
auc = evaluator.evaluate(predictions)
print("AUC:", auc)

AUC: 0.8431148373983745
